In [68]:
# Import libraries necesarry for web scraping with selenium
#import selenium
from selenium import webdriver
#from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
#from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.service import Service as ChromeService
#import time
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver import ActionChains

from selenium.webdriver import Firefox
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager

import requests
from bs4 import BeautifulSoup
import json
import PyPDF2
import io
import re
import pandas as pd
# Import necessary libraries


Here are the Websites I am thinking of using:

https://cde.ucr.cjis.gov/LATEST/webapp/#/pages/docApi

https://ucr.fbi.gov/crime-in-the-u.s/2019/crime-in-the-u.s.-2019/topic-pages/tables/table-5

https://www.icpsr.umich.edu/web/NACJD/studies/38604/summary

https://bjs.ojp.gov/library/publications/criminal-victimization-2022

https://endrapeoncampus.org/map-and-tool/

https://ope.ed.gov/campussafety/#/compare/confirm

https://www.prri.org/research/2020-census-of-american-religion/

https://www.statista.com/statistics/221454/share-of-religious-americans-by-state/

BIG 12 (for 2024 group) FIRE & SAFETY REPORTS:

BYU https://police.byu.edu/2023-asr-provo  (page 9)

Baylor https://www.baylor.edu/risk/doc.php/399779.pdf (page 12)

UCF https://police.ucf.edu/sites/default/files/2023%20Annual%20Security%20%26%20Fire%20Safety%20Report.pdf (page 78)

Cinncinati https://www.uc.edu/content/dam/refresh/publicsafety-62/docs/ucpd/2023-annual-security-report.pdf (page 34)

Houston http://m2s-conf.uh.edu/police/records-reports/annualreport/2023-uh-annual-security-and-fire-safety-report.pdf (page 39)

Iowa https://www.police.iastate.edu/wp-content/uploads/2023/09/2022-Report-Final.pdf (page 6)

KU https://civilrights.ku.edu/sites/civilrights/files/documents/Reports/University%20of%20Kansas%20Annual%20Security%20and%20Fire%20Safety%20Report-2023-KLETC.pdf (page 60)

K-state https://www.k-state.edu/report/clery/reports/KSUCleryReport2023.pdf (page 13)

OSU https://safety.okstate.edu/police/documents/annual-security-reports/2023-annual-safety-report.pdf (page 37)

TCU https://police.tcu.edu/wp-content/uploads/2023/09/TCU-Annual-Security-Report-Fire-Safety-Report_2023.v1.pdf (page 80)

Texas Tech https://www.depts.ttu.edu/clery/reports/2023LubbockASRfinal.pdf (page 107)

West Virginia https://police.wvu.edu/files/d/a1a0a6fe-444e-4353-8e92-7dcbeb00cd27/2023-annual-security-and-fire-safety-report-with-appendix.pdf (page 36)

In [2]:
import camelot

In [341]:
BYU_table = camelot.read_pdf('BYU_report.pdf', pages='9,10', flavor='stream')
Baylor_tables = camelot.read_pdf('Baylor_report.pdf', pages='12,13', flavor='stream')
Baylor_table_VAWA = camelot.read_pdf('Baylor_report.pdf', pages='13', flavor='stream', table_regions=['0,400,800,0'])
Cinn_table = camelot.read_pdf('Cinn_report.pdf', pages='34', flavor='stream')
Iowa_table = camelot.read_pdf('Iowa_report.pdf', pages='6', flavor='stream')
Kstate_table = camelot.read_pdf('Kstate_report.pdf', pages='13', flavor='stream')
KU_tables = camelot.read_pdf('KU_report.pdf', pages='60,61', flavor='stream')
OSU_tables = camelot.read_pdf('OSU_report.pdf', pages='37,38', flavor='stream')
TCU_table = camelot.read_pdf('TCU_report.pdf', pages='80', flavor='stream')
Ttech_table = camelot.read_pdf('Ttech_report.pdf', pages='107', flavor='stream')
UCF_tables = camelot.read_pdf('UCF_report.pdf', pages='78,79,80', flavor='stream')
UH_table = camelot.read_pdf('UH_report.pdf', pages='39', flavor='stream')
WV_table = camelot.read_pdf('WV_report.pdf', pages='36', flavor='stream')

In [398]:
BYU_table[0].df
# take only rows we need
BYU_sexOff = BYU_table[0].df.iloc[[1,9,10,11,12]]
BYU_VAWA = BYU_table[1].df.iloc[[2,4,5,6]]
# take out columns we don't need
BYU_sexOff = BYU_sexOff.drop([2,4,6], axis=1)
BYU_VAWA = BYU_VAWA.drop([2,4,6], axis=1)
# reset the index
BYU_sexOff = BYU_sexOff.reset_index(drop=True)
BYU_VAWA = BYU_VAWA.reset_index(drop=True)
# make the first row the column names
BYU_sexOff.columns = BYU_sexOff.iloc[0]
BYU_VAWA.columns = BYU_VAWA.iloc[0]
# drop the first row
BYU_sexOff = BYU_sexOff.drop([0])
BYU_VAWA = BYU_VAWA.drop([0])
# reset the index
BYU_sexOff = BYU_sexOff.reset_index(drop=True)
BYU_VAWA = BYU_VAWA.reset_index(drop=True)
# Adding the year counts together
BYU_sexOff['2020 Total'] = BYU_sexOff['2020'].astype(int).sum(axis = 1)
BYU_sexOff['2021 Total'] = BYU_sexOff['2021'].astype(int).sum(axis = 1)
BYU_sexOff['2022 Total'] = BYU_sexOff['2022'].astype(int).sum(axis = 1)
BYU_VAWA['2020 Total'] = BYU_VAWA['2020'].astype(int).sum(axis = 1)
BYU_VAWA['2021 Total'] = BYU_VAWA['2021'].astype(int).sum(axis = 1)
BYU_VAWA['2022 Total'] = BYU_VAWA['2022'].astype(int).sum(axis = 1)
# Add the school name
BYU_sexOff['School'] = 'BYU'
BYU_VAWA['School'] = 'BYU'
# Keep only the columns we want
BYU_sexOff = BYU_sexOff[['Year Reported','School','2020 Total','2021 Total','2022 Total']]
BYU_VAWA = BYU_VAWA[['Year Reported','School','2020 Total','2021 Total','2022 Total']]
# Rename Year Reported to Offense
BYU_sexOff = BYU_sexOff.rename(columns={'Year Reported': 'Offense'})
BYU_VAWA = BYU_VAWA.rename(columns={'Year Reported': 'Offense'})

# Combine the two BYU tables
BYU_sexOff = BYU_sexOff.append(BYU_VAWA)
# Reset the index
BYU_sexOff = BYU_sexOff.reset_index(drop=True)
BYU_sexOff

C:\Users\hermi\AppData\Local\Temp\ipykernel_17888\2428548272.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  BYU_sexOff = BYU_sexOff.append(BYU_VAWA)


,Offense,School,2020 Total,2021 Total,2022 Total
0,Rape,BYU,4,7,6
1,Fondling,BYU,10,13,17
2,Incest,BYU,0,0,0
3,Statutory Rape,BYU,0,0,0
4,Domestic Violence,BYU,0,2,1
5,Dating Violence,BYU,5,5,17
6,Stalking,BYU,14,6,16


In [399]:
Baylor_tables[0].df
# Only take row 16 and below
Baylor_sexOff = Baylor_tables[0].df[16:]
Baylor_sexOff = Baylor_sexOff.drop([19], axis=0)
# reset the index
Baylor_sexOff = Baylor_sexOff.reset_index(drop=True)
# Remove the 0 and 2 columns
Baylor_sexOff = Baylor_sexOff.drop([0,2], axis=1)
# Add a row with custom values
data = {1:'Statutory Rape', 3:'0',4:'0',5:'0',6:'0',7:'0',8:'0'}
# Append the row to the dataframe
Baylor_sexOff = Baylor_sexOff.append(data, ignore_index=True)
# only keep columns 1,6
Baylor_sexOff = Baylor_sexOff[[1,6]] 
# Rename the columns
Baylor_sexOff = Baylor_sexOff.rename(columns={1: 'Offense', 6: '2020 Total'}) 
# remove the first row
Baylor_sexOff = Baylor_sexOff.drop([0], axis=0) 

Baylor_sexOff['School'] = 'BU'
Baylor_sexOff['2021 Total'] = ['20','10','0','0']
Baylor_sexOff['2022 Total'] = ['22','8','0','0']

# Reorder the columns
Baylor_sexOff = Baylor_sexOff[['Offense','School','2020 Total','2021 Total','2022 Total']]
# Reset the index
Baylor_sexOff = Baylor_sexOff.reset_index(drop=True)
Baylor_sexOff

# Set up Baylor VAWA
Baylor_VAWA = Baylor_table_VAWA[0].df[[1,2,6]]
# Remove rows 0 5 and 7
Baylor_VAWA = Baylor_VAWA.drop([0,5,7], axis=0)
# Fill the First column with the offense
Baylor_VAWA[1] = ['Domestic Violence','Domestic Violence','Domestic Violence','Dating Violence','Dating Violence','Dating Violence','Stalking','Stalking','Stalking']
# Rename the columns
Baylor_VAWA = Baylor_VAWA.rename(columns={1: 'Offense', 2: 'Year', 6: 'Total'})
# Pivot the table, keeping Offense in as a column
Baylor_VAWA = Baylor_VAWA.pivot(index='Offense', columns='Year', values='Total')
# Reset the index
Baylor_VAWA = Baylor_VAWA.reset_index()
# Add the school name
Baylor_VAWA['School'] = 'BU'
# Reorder the columns
Baylor_VAWA = Baylor_VAWA[['Offense','School','2020','2021','2022']]
# Rename the columns
Baylor_VAWA = Baylor_VAWA.rename(columns={'2020': '2020 Total','2021': '2021 Total','2022': '2022 Total'})
# Rename the index
Baylor_VAWA = Baylor_VAWA.rename_axis(None, axis=1)

# Combine the two Baylor tables
Baylor_sexOff = Baylor_sexOff.append(Baylor_VAWA, ignore_index=True)
# switch rows 4 and 5
Baylor_sexOff.iloc[[4,5]] = Baylor_sexOff.iloc[[5,4]] 

Baylor_sexOff

C:\Users\hermi\AppData\Local\Temp\ipykernel_17888\535639888.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Baylor_sexOff = Baylor_sexOff.append(data, ignore_index=True)
C:\Users\hermi\AppData\Local\Temp\ipykernel_17888\535639888.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Baylor_sexOff = Baylor_sexOff.append(Baylor_VAWA, ignore_index=True)


,Offense,School,2020 Total,2021 Total,2022 Total
0,Rape,BU,13,20,22
1,Fondling,BU,13,10,8
2,Incest,BU,0,0,0
3,Statutory Rape,BU,0,0,0
4,Domestic Violence,BU,7,8,13
5,Dating Violence,BU,4,5,0
6,Stalking,BU,17,23,40


In [395]:
Cinn_table[0].df
# Keep rows 3, 6-9: Sex offenses, 15-17: Stalking/dating violence
Cinn_sexOff = Cinn_table[0].df.iloc[[3,6,7,8,9,15,16,17]]
# Reset the index
Cinn_sexOff = Cinn_sexOff.reset_index(drop=True)
# Make first row the columns
Cinn_sexOff.columns = Cinn_sexOff.iloc[0]
# Drop the first row
Cinn_sexOff = Cinn_sexOff.drop([0])
# Reset the index
Cinn_sexOff = Cinn_sexOff.reset_index(drop=True)
# Remove all the * from row 1
Cinn_sexOff.iloc[0,] = Cinn_sexOff.iloc[0,].str.replace('*','')
# Add the school name
Cinn_sexOff['School'] = 'CU'
# Sum the columns
Cinn_sexOff['2020 Total'] = Cinn_sexOff['2020'].astype(int).sum(axis = 1)
Cinn_sexOff['2021 Total'] = Cinn_sexOff['2021'].astype(int).sum(axis = 1)
Cinn_sexOff['2022 Total'] = Cinn_sexOff['2022'].astype(int).sum(axis = 1)
# Keep only the columns we want
Cinn_sexOff = Cinn_sexOff[['Offense','School','2020 Total','2021 Total','2022 Total']]
# Swap rows 3 and 4
Cinn_sexOff.iloc[[2,3]] = Cinn_sexOff.iloc[[3,2]]
Cinn_sexOff
#Cinn_table[0].df

C:\Users\hermi\AppData\Local\Temp\ipykernel_17888\1746277527.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Cinn_sexOff.iloc[0,] = Cinn_sexOff.iloc[0,].str.replace('*','')


,Offense,School,2020 Total,2021 Total,2022 Total
0,Rape,CU,20,34,52
1,Fondling,CU,7,20,19
2,Incest,CU,0,0,0
3,Statutory Rape,CU,0,0,0
4,Domestic Violence,CU,8,8,12
5,Dating Violence,CU,8,13,13
6,Stalking,CU,23,15,16


In [408]:
Iowa_table[0].df
# Keep idex rows 0,1, 8-11
Iowa_sexOff = Iowa_table[0].df.iloc[[0,1,8,9,10,11,36,38,39]]
# Reset the index
Iowa_sexOff = Iowa_sexOff.reset_index(drop=True)
# Keep columns 0,16,17,18
Iowa_sexOff = Iowa_sexOff[[0,16,17,18]]
# Drop row 0 and 1
Iowa_sexOff = Iowa_sexOff.drop([0,1], axis=0)
# Reset the index
Iowa_sexOff = Iowa_sexOff.reset_index(drop=True)
# Rename the columns
Iowa_sexOff = Iowa_sexOff.rename(columns={0: 'Offense', 16: '2020 Total', 17: '2021 Total', 18: '2022 Total'})
# Add the school name
Iowa_sexOff['School'] = 'Iowa'
# Reorder the columns
Iowa_sexOff = Iowa_sexOff[['Offense','School','2020 Total','2021 Total','2022 Total']]
# Rename Offense in row 4 and 5
Iowa_sexOff.iloc[4,0] = 'Dating Violence'
Iowa_sexOff.iloc[5,0] = 'Domestic Violence'
# Swap rows 4 and 5
Iowa_sexOff.iloc[[4,5]] = Iowa_sexOff.iloc[[5,4]]

Iowa_sexOff

,Offense,School,2020 Total,2021 Total,2022 Total
0,Rape,Iowa,10,22,18
1,Fondling,Iowa,4,7,6
2,Incest,Iowa,0,0,0
3,Statutory Rape,Iowa,0,0,0
4,Domestic Violence,Iowa,5,2,6
5,Dating Violence,Iowa,8,5,6
6,Stalking,Iowa,22,32,21


In [382]:
Kstate_table = camelot.read_pdf('Kstate_report.pdf', pages='13', flavor='stream', table_regions=['0,600,800,0'])

In [410]:
Kstate_table[0].df
# Keep idex rows 9, 17-20: Sex offenses, 22-24: Stalking/dating violence
Kstate_sexOff = Kstate_table[0].df.iloc[[17,18,19,20,22,23,24]]
# Rename the columns
Kstate_sexOff.columns = ['Offense','2020','2020','2020','2020','2021','2021','2021','2021','2022','2022','2022','2022']
# Make yearly totals
Kstate_sexOff['2020 Total'] = Kstate_sexOff['2020'].astype(int).sum(axis = 1)
Kstate_sexOff['2021 Total'] = Kstate_sexOff['2021'].astype(int).sum(axis = 1)
Kstate_sexOff['2022 Total'] = Kstate_sexOff['2022'].astype(int).sum(axis = 1)
# Make school column
Kstate_sexOff['School'] = 'KSU'
# Keep only the columns we want
Kstate_sexOff = Kstate_sexOff[['Offense','School','2020 Total','2021 Total','2022 Total']]
# Drop the first row
#Kstate_sexOff = Kstate_sexOff.drop([0], axis=0)
# Reset the index
Kstate_sexOff = Kstate_sexOff.reset_index(drop=True)

Kstate_sexOff

C:\Users\hermi\AppData\Local\Temp\ipykernel_17888\1603668424.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Kstate_sexOff['2020 Total'] = Kstate_sexOff['2020'].astype(int).sum(axis = 1)
C:\Users\hermi\AppData\Local\Temp\ipykernel_17888\1603668424.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Kstate_sexOff['2021 Total'] = Kstate_sexOff['2021'].astype(int).sum(axis = 1)
C:\Users\hermi\AppData\Local\Temp\ipykernel_17888\1603668424.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy 

,Offense,School,2020 Total,2021 Total,2022 Total
0,Rape,KSU,5,14,26
1,Fondling,KSU,1,18,6
2,Incest,KSU,0,0,0
3,Statutory Rape,KSU,0,0,0
4,Domestic Violence,KSU,5,3,7
5,Dating Violence,KSU,0,12,5
6,Stalking,KSU,6,18,8


In [222]:
KU_tables[0].df
# Keep idex rows 8-19
KU_sexOff = KU_table[0].df.iloc[[8,9,10,11,12,13,14,15,16,17,18,19]]
# Reset the index
KU_sexOff = KU_sexOff.reset_index(drop=True)
# Pivot the table, keeping columns 0 the same, and moving columns 1 to the rows


#KU_sexOff

In [414]:
# Create a data frame from scratch that matches the BYU_sexOff data frame for the KU data
KU_scratch = pd.DataFrame(columns=['Offense','School','2020 Total','2021 Total','2022 Total'])
# Add this row to the data frame
KU_data = {'Offense':'Rape', 'School': 'KU', '2020 Total':'0', '2021 Total':'0', '2022 Total':'0'}
KU_scratch = KU_scratch.append(KU_data, ignore_index=True)
KU_data = {'Offense':'Fondling', 'School': 'KU', '2020 Total':'0', '2021 Total':'0', '2022 Total':'1'}
KU_scratch = KU_scratch.append(KU_data, ignore_index=True)
KU_data = {'Offense':'Incest', 'School': 'KU', '2020 Total':'0', '2021 Total':'0', '2022 Total':'0'}
KU_scratch = KU_scratch.append(KU_data, ignore_index=True)
KU_data = {'Offense':'Statutory Rape', 'School': 'KU', '2020 Total':'0', '2021 Total':'0', '2022 Total':'0'}
KU_scratch = KU_scratch.append(KU_data, ignore_index=True)
KU_data = {'Offense':'Domestic Violence', 'School': 'KU', '2020 Total':'0', '2021 Total':'0', '2022 Total':'0'}
KU_scratch = KU_scratch.append(KU_data, ignore_index=True)
KU_data = {'Offense':'Dating Violence', 'School': 'KU', '2020 Total':'0', '2021 Total':'0', '2022 Total':'0'}
KU_scratch = KU_scratch.append(KU_data, ignore_index=True)
KU_data = {'Offense':'Stalking', 'School': 'KU', '2020 Total':'0', '2021 Total':'0', '2022 Total':'1'}
KU_scratch = KU_scratch.append(KU_data, ignore_index=True)

KU_sexOff = KU_scratch

C:\Users\hermi\AppData\Local\Temp\ipykernel_17888\1361014399.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  KU_scratch = KU_scratch.append(KU_data, ignore_index=True)
C:\Users\hermi\AppData\Local\Temp\ipykernel_17888\1361014399.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  KU_scratch = KU_scratch.append(KU_data, ignore_index=True)
C:\Users\hermi\AppData\Local\Temp\ipykernel_17888\1361014399.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  KU_scratch = KU_scratch.append(KU_data, ignore_index=True)
C:\Users\hermi\AppData\Local\Temp\ipykernel_17888\1361014399.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  KU_scratch = K

In [417]:
OSU_tables = camelot.read_pdf('OSU_report.pdf', pages='37,38', flavor='stream')

In [418]:
OSU_tables[1].df

,0,1,2,3,4,5,6
0,with the Uniform Crime Reporting procedures an...,,,,,,
1,,2022,0,1,0,1,0
2,MOTOR VEHICLE THEFT,2021,4,1,0,5,0
3,,2020,0,1,0,1,0
4,,2022,0,0,0,0,0
5,ARSON,2021,0,0,0,0,0
6,,2020,0,0,0,0,0
7,,2022,7,0,0,7,4
8,DATING VIOLENCE,2021,2,0,0,2,2
9,,2020,4,0,0,4,3


In [422]:
OSU_tables[0].df
# Keep idex rows 0 9-20
OSU_sexOff = OSU_tables[0].df.iloc[[9,10,11,12,13,14,15,16,17,18,19,20]]
OSU_VAWA = OSU_tables[1].df.iloc[[7,8,9,10,11,12,13,14,15]]
# Keep index columns 0,1,5
OSU_sexOff = OSU_sexOff[[0,1,5]]
OSU_VAWA = OSU_VAWA[[0,1,5]]
# Fill in the rest of column 0
OSU_sexOff['Offense'] = ['Rape', 'Rape', 'Rape', 'Fondling', 'Fondling', 'Fondling', 'Incest', 'Incest', 'Incest', 'Statutory Rape', 'Statutory Rape', 'Statutory Rape']
OSU_VAWA['Offense'] = ['Dating Violence', 'Dating Violence', 'Dating Violence', 'Domestic Violence', 'Domestic Violence', 'Domestic Violence', 'Stalking', 'Stalking', 'Stalking']
# Replace Column 0 with the Offense column
OSU_sexOff = OSU_sexOff[['Offense',1,5]]
OSU_VAWA = OSU_VAWA[['Offense',1,5]]
# Pivot the tables
OSU_sexOff = OSU_sexOff.pivot(columns=1, index='Offense')
OSU_sexOff.columns = OSU_sexOff.columns.droplevel()
OSU_sexOff = OSU_sexOff.reset_index()
OSU_sexOff = OSU_sexOff.rename_axis(None, axis=1)

OSU_VAWA = OSU_VAWA.pivot(columns=1, index='Offense')
OSU_VAWA.columns = OSU_VAWA.columns.droplevel()
OSU_VAWA = OSU_VAWA.reset_index()
OSU_VAWA = OSU_VAWA.rename_axis(None, axis=1)

# Add the school name
OSU_sexOff['School'] = 'OSU'
OSU_VAWA['School'] = 'OSU'
# Rename the columns and Reorder the columns
OSU_sexOff = OSU_sexOff.rename(columns={'2020': '2020 Total', '2021': '2021 Total', '2022': '2022 Total'})
OSU_VAWA = OSU_VAWA.rename(columns={'2020': '2020 Total', '2021': '2021 Total', '2022': '2022 Total'})
OSU_sexOff = OSU_sexOff[['Offense','School','2020 Total','2021 Total','2022 Total']]
OSU_VAWA = OSU_VAWA[['Offense','School','2020 Total','2021 Total','2022 Total']]
# Reorder the rows
OSU_sexOff = OSU_sexOff.iloc[[2,0,1,3]]
OSU_VAWA = OSU_VAWA.iloc[[1,0,2]]
# Reset the index
OSU_sexOff = OSU_sexOff.reset_index(drop=True)
OSU_VAWA = OSU_VAWA.reset_index(drop=True)
# Combine the two OSU tables
OSU_sexOff = OSU_sexOff.append(OSU_VAWA, ignore_index=True)

OSU_sexOff

C:\Users\hermi\AppData\Local\Temp\ipykernel_17888\2494744416.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  OSU_sexOff = OSU_sexOff.append(OSU_VAWA, ignore_index=True)


,Offense,School,2020 Total,2021 Total,2022 Total
0,Rape,OSU,4,15,12
1,Fondling,OSU,6,4,0
2,Incest,OSU,0,0,0
3,Statutory Rape,OSU,0,0,0
4,Domestic Violence,OSU,8,6,3
5,Dating Violence,OSU,4,2,7
6,Stalking,OSU,9,12,16


In [316]:
TCU_table[0].df
# Keep rows 11-23
TCU_sexOff = TCU_table[0].df.iloc[[11,12,13,14,15,16,17,18,19,20,21,22]]
# Dop columns 3 and 5
TCU_sexOff = TCU_sexOff.drop([3,5], axis=1)
# Fill in the rest of column 0
TCU_sexOff['Offense'] = ['Rape', 'Rape', 'Rape', 'Fondling', 'Fondling', 'Fondling', 'Incest', 'Incest', 'Incest', 'Statutory Rape', 'Statutory Rape', 'Statutory Rape']
# Replace Column 0 with the Offense column
TCU_sexOff = TCU_sexOff[['Offense',1,2,4]]
# Pivot the table
TCU_sexOff = TCU_sexOff.pivot(columns=1, index='Offense')
TCU_sexOff.columns = TCU_sexOff.columns.droplevel()
TCU_sexOff = TCU_sexOff.reset_index()
TCU_sexOff = TCU_sexOff.rename_axis(None, axis=1)
# Remove the ^ from the columns
TCU_sexOff.iloc[0,2] = '8'
# Make yearly totals
TCU_sexOff['2020 Total'] = TCU_sexOff['2020'].astype(int).sum(axis = 1)
TCU_sexOff['2021 Total'] = TCU_sexOff['2021'].astype(int).sum(axis = 1)
TCU_sexOff['2022 Total'] = TCU_sexOff['2022'].astype(int).sum(axis = 1)
# Add the school name
TCU_sexOff['School'] = 'TCU'
# Keep only the columns we want
TCU_sexOff = TCU_sexOff[['Offense','School','2020 Total','2021 Total','2022 Total']]
# Reorder the rows
TCU_sexOff = TCU_sexOff.iloc[[2,0,1,3]]
# Reset the index
TCU_sexOff = TCU_sexOff.reset_index(drop=True)


TCU_sexOff

,Offense,School,2020 Total,2021 Total,2022 Total
0,Rape,TCU,17,22,12
1,Fondling,TCU,4,8,4
2,Incest,TCU,0,0,0
3,Statutory Rape,TCU,0,0,0


In [317]:
Ttech_table_22 = camelot.read_pdf('Ttech_report.pdf', pages='107', flavor='stream')
Ttech_table_21 = camelot.read_pdf('Ttech_report.pdf', pages='109', flavor='stream')
Ttech_table_20 = camelot.read_pdf('Ttech_report.pdf', pages='111', flavor='stream')

In [329]:
Ttech_table_22[0].df
# Keep rows 10-13
Ttech_sexOff_22 = Ttech_table_22[0].df.iloc[[10,11,12,14]]
# Keep columns 0 and 5
Ttech_sexOff_22 = Ttech_sexOff_22[[0,5]]

# Keep rows 10-13
Ttech_sexOff_21 = Ttech_table_21[0].df.iloc[[10,11,12,14]]
# Keep columns 0 and 5
Ttech_sexOff_21 = Ttech_sexOff_21[[0,5]]

# Keep rows 10-13
Ttech_sexOff_20 = Ttech_table_20[0].df.iloc[[10,11,12,14]]
# Keep columns 0 and 5
Ttech_sexOff_20 = Ttech_sexOff_20[[0,5]]

Ttech_sexOff_20.rename(columns={0:'Offense', 5:'2020 Total'}, inplace=True) 
Ttech_sexOff_22.rename(columns={0:'Offense', 5:'2022 Total'}, inplace=True)
Ttech_sexOff_21.rename(columns={0:'Offense', 5:'2021 Total'}, inplace=True)
# Join the three tables together on the Offense column
Ttech_sexOff = Ttech_sexOff_20.merge(Ttech_sexOff_21, on='Offense', how='outer').merge(Ttech_sexOff_22, on='Offense', how='outer')

# Add the school name
Ttech_sexOff['School'] = 'TTU'
# Rename the rows
Ttech_sexOff['Offense'] = ['Rape','Fondling','Incest','Statutory Rape']
# Reorder the columns
Ttech_sexOff = Ttech_sexOff[['Offense','School','2020 Total','2021 Total','2022 Total']]

Ttech_sexOff

,Offense,School,2020 Total,2021 Total,2022 Total
0,Rape,TTU,7,26,23
1,Fondling,TTU,7,19,8
2,Incest,TTU,0,0,0
3,Statutory Rape,TTU,0,0,0


In [340]:
UCF_tables[0].df
# Keep rows 8-11
UCF_sexOff_22 = UCF_tables[0].df.iloc[[8,9,10,11]]
UCF_sexOff_21 = UCF_tables[1].df.iloc[[8,9,10,11]]
UCF_sexOff_20 = UCF_tables[2].df.iloc[[8,9,10,11]]
# Drop columns 2 and 4
UCF_sexOff_20 = UCF_sexOff_20.drop([2,4], axis=1)
UCF_sexOff_22 = UCF_sexOff_22.drop([2,4], axis=1)
UCF_sexOff_21 = UCF_sexOff_21.drop([2,4], axis=1)
# Rename columns
UCF_sexOff_22.columns = ['Offense','2022', '2022']
UCF_sexOff_21.columns = ['Offense','2021', '2021']
UCF_sexOff_20.columns = ['Offense','2020', '2020']
# Make yearly totals
UCF_sexOff_20['2020 Total'] = UCF_sexOff_20['2020'].astype(int).sum(axis = 1)
UCF_sexOff_21['2021 Total'] = UCF_sexOff_21['2021'].astype(int).sum(axis = 1)
UCF_sexOff_22['2022 Total'] = UCF_sexOff_22['2022'].astype(int).sum(axis = 1)
# Keep the columns we want
UCF_sexOff_20 = UCF_sexOff_20[['Offense','2020 Total']]
UCF_sexOff_21 = UCF_sexOff_21[['Offense','2021 Total']]
UCF_sexOff_22 = UCF_sexOff_22[['Offense','2022 Total']]
# Join the tables together
UCF_sexOff = UCF_sexOff_20.merge(UCF_sexOff_21, on='Offense', how='outer').merge(UCF_sexOff_22, on='Offense', how='outer')
# Add the school name
UCF_sexOff['School'] = 'UCF'
# Reorder the columns
UCF_sexOff = UCF_sexOff[['Offense','School','2020 Total','2021 Total','2022 Total']]



UCF_sexOff

,Offense,School,2020 Total,2021 Total,2022 Total
0,Rape,UCF,8,16,14
1,Fondling,UCF,7,8,8
2,Incest,UCF,0,0,0
3,Statutory Rape,UCF,0,0,1


In [296]:
UH_table[0].df
# Keep rows  13-26
UH_sexOff = UH_table[0].df.iloc[[14,15,16,17,18,19,20,21,22,23,24,25]]
# Keep columns 0,1,6
UH_sexOff = UH_sexOff[[0,1,6]]

UH_sexOff['Offense'] = ['Rape', 'Rape', 'Rape', 'Fondling', 'Fondling', 'Fondling', 'Incest', 'Incest', 'Incest', 'Statutory Rape', 'Statutory Rape', 'Statutory Rape']
# Replace Column 0 with the Offense column
UH_sexOff = UH_sexOff[['Offense',1,6]]
# Pivot the table
UH_sexOff = UH_sexOff.pivot(columns=1, index='Offense')
UH_sexOff.columns = UH_sexOff.columns.droplevel()
UH_sexOff = UH_sexOff.reset_index()
UH_sexOff = UH_sexOff.rename_axis(None, axis=1)

# Add the school name
UH_sexOff['School'] = 'UH'
# Rename the columns and Reorder the columns
UH_sexOff = UH_sexOff.rename(columns={'2020': '2020 Total', '2021': '2021 Total', '2022': '2022 Total'})
UH_sexOff = UH_sexOff[['Offense','School','2020 Total','2021 Total','2022 Total']]
# Reorder the rows
UH_sexOff = UH_sexOff.iloc[[2,0,1,3]]
# Reset the index
UH_sexOff = UH_sexOff.reset_index(drop=True)
UH_sexOff

,Offense,School,2020 Total,2021 Total,2022 Total
0,Rape,UH,5,12,13
1,Fondling,UH,4,5,1
2,Incest,UH,0,0,0
3,Statutory Rape,UH,1,0,0


In [291]:
WV_table[0].df 
# Keep rows 15-21
WV_sexOff = WV_table[0].df.iloc[[15,16,17,18,19,20]]
WV_sexOff.iloc[0,3] = '19'
WV_sexOff.iloc[3,2] = '7'
# Keep only columns 0,1,6
WV_sexOff = WV_sexOff[[0,1,6]]

WV_sexOff['Offense'] = ['Rape', 'Rape', 'Rape', 'Fondling', 'Fondling', 'Fondling']
# Replace Column 0 with the Offense column
WV_sexOff = WV_sexOff[['Offense',1,6]]

# Pivot the table
WV_sexOff = WV_sexOff.pivot(columns=1, index='Offense')
WV_sexOff.columns = WV_sexOff.columns.droplevel()
WV_sexOff = WV_sexOff.reset_index()
WV_sexOff = WV_sexOff.rename_axis(None, axis=1)

# Add the school name
WV_sexOff['School'] = 'WV'
# Rename the columns and Reorder the columns
WV_sexOff = WV_sexOff.rename(columns={'2020': '2020 Total', '2021': '2021 Total', '2022': '2022 Total'})
WV_sexOff = WV_sexOff[['Offense','School','2020 Total','2021 Total','2022 Total']]
# Fill in the empty Data
WV_data = {'Offense':'Incest', 'School': 'WV', '2020 Total':'0', '2021 Total':'0', '2022 Total':'0'}
WV_sexOff = WV_sexOff.append(WV_data, ignore_index=True)
WV_data = {'Offense':'Statutory Rape', 'School': 'WV', '2020 Total':'0', '2021 Total':'0', '2022 Total':'0'}
WV_sexOff = WV_sexOff.append(WV_data, ignore_index=True)
# Reorder the rows
WV_sexOff = WV_sexOff.iloc[[1,0,2,3]]
# Reset the index
WV_sexOff = WV_sexOff.reset_index(drop=True)

WV_sexOff

C:\Users\hermi\AppData\Local\Temp\ipykernel_17888\3118606000.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  WV_sexOff.iloc[0,3] = '19'
C:\Users\hermi\AppData\Local\Temp\ipykernel_17888\3118606000.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  WV_sexOff.iloc[3,2] = '7'
C:\Users\hermi\AppData\Local\Temp\ipykernel_17888\3118606000.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  WV_sexOff = WV_sexOff.append(WV_data, ignore_index=True)
C:\Users\hermi\AppData\Local\Temp\ipykernel_17888\3118606000.py:28: FutureWarning: The 

,Offense,School,2020 Total,2021 Total,2022 Total
0,Rape,WV,9,17,21
1,Fondling,WV,4,12,17
2,Incest,WV,0,0,0
3,Statutory Rape,WV,0,0,0


In [393]:

# Creat the Master Table by Joining all the sexOff tables together

# combine the dataframes
master_table = pd.concat([BYU_sexOff, Baylor_sexOff, Cinn_sexOff, Iowa_sexOff, Kstate_sexOff, KU_sexOff, OSU_sexOff, TCU_sexOff, Ttech_sexOff,UCF_sexOff,UH_sexOff,WV_sexOff], ignore_index=True)

# print the master table
print(master_table)


           Offense School 2020 Total 2021 Total 2022 Total
0             Rape    BYU          4          7          6
1         Fondling    BYU         10         13         17
2           Incest    BYU          0          0          0
3   Statutory Rape    BYU          0          0          0
4             Rape     BU         13         20         22
5         Fondling     BU         13         10          8
6           Incest     BU          0          0          0
7   Statutory Rape     BU          0          0          0
8             Rape     CU         20         34         52
9         Fondling     CU          7         20         19
10          Incest     CU          0          0          0
11  Statutory Rape     CU          0          0          0
12            Rape   Iowa         10         22         18
13        Fondling   Iowa          4          7          6
14          Incest   Iowa          0          0          0
15  Statutory Rape   Iowa          0          0         